In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys, os
from math import ceil
import numpy as np
import pandas as pd
import seaborn as sb
from scipy.spatial.transform import Rotation as R
from scipy import stats
import mdtraj as md
import tables as tb
import nglview as nv
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

up_ver = "upside2-md"
upside_utils_dir = os.path.expanduser("~/{}/py".format(up_ver))
sys.path.append(upside_utils_dir)
import mdtraj_upside as mu

Very Important: All distances are in nanometers for MDTraj


In [3]:
mpl.rcParams['font.size'] = 18
mpl.rcParams['axes.labelsize'] = 'medium'
mpl.rcParams['ytick.labelsize'] = 'medium'
mpl.rcParams['xtick.labelsize'] = 'medium'
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['xtick.minor.width'] = 2
mpl.rcParams['ytick.major.width'] = 2
mpl.rcParams['ytick.minor.width'] = 2
mpl.rcParams['font.sans-serif'] = "Arial"
mpl.rcParams['font.family'] = "sans-serif"
mpl.rcParams['svg.fonttype'] = 'none'

In [4]:
base_dir = "/home/okleinmann/scratch-midway2/"
up_indir = base_dir + "up_input/"
pdb_id = "1hk9_model"
n_rep = 20

output_dir = base_dir + "runs/replex7/"

init_fn = up_indir + "{}.initial.npy".format(pdb_id)
fasta_fn = up_indir + "{}.fasta".format(pdb_id)
chain_breaks_fn = up_indir + "{}.chain_breaks".format(pdb_id)

traj = mu.load_upside_traj(init_fn, from_init=True, fasta_fn=fasta_fn, chain_breaks_fn=chain_breaks_fn)

acid_sele = traj.top.select("(resname ASP or resname GLU) and name CA")
base_sele = traj.top.select("(resname LYS or resname ARG) and name CA")

In [ ]:
print(traj)

In [ ]:
view = nv.NGLWidget()
view.add_trajectory(traj)
view.clear_representations()
view.add_cartoon(color='gray')
view.add_spacefill(selection=acid_sele, color='red')
view.add_spacefill(selection=base_sele, color='blue')

view.camera = 'orthographic'
view.parameters = {
    "clipNear": -100, "clipFar": 1000,
    "fogNear": 0, "fogFar": 1000,
}
view

In [ ]:
dssp = md.compute_dssp(traj[0])[0]
for i, ss in enumerate(dssp):
    print (i, ss)

In [ ]:
restr_list = []
for i in range(traj.n_chains):
    restr_list.append([i*102, i*102+66])

In [ ]:
restr_str = []
a_sele = []
for grp in restr_list:
    restr_str.append("{}-{}".format(grp[0], grp[1]))
    
    # for visualization check
    a_sele.append(traj.top.select("resid {} to {}".format(grp[0], grp[1])))
a_sele = np.concatenate(a_sele)
print (restr_str)

In [ ]:
# Visualization check
view = nv.NGLWidget()
view.add_trajectory(traj[0])
view.clear_representations()
view.add_backbone(selection=a_sele, color="magenta")
view.camera = 'orthographic'
view

In [1]:
traj_fn = output_dir + "{}.{}.up".format(pdb_id, 0)
traj = mu.load_upside_traj(traj_fn, stride=1)

restr_list = []
for i in range(traj.n_chains):
    restr_list.append([i*102, i*102+66])

a_sele = []
for grp in restr_list:
    a_sele.append(traj.top.select("resid {} to {}".format(grp[0], grp[1])))
a_sele = np.concatenate(a_sele)
traj.superpose(traj[0], atom_indices=a_sele)

acid_sele = traj.top.select("(resname ASP or resname GLU) and name CA")
base_sele = traj.top.select("(resname LYS or resname ARG) and name CA")

view = nv.NGLWidget()
view.add_trajectory(traj[:])
view.clear_representations()
view.add_cartoon(color='gray')
view.add_spacefill(selection=acid_sele, color='red')
view.add_spacefill(selection=base_sele, color='blue')

view.camera = 'orthographic'
view.parameters = {
    "clipNear": -100, "clipFar": 1000,
    "fogNear": 0, "fogFar": 1000,
}
view

NameError: name 'output_dir' is not defined

In [ ]:
xyz = traj.xyz

In [ ]:
xyz[:, acid_sele, :]

# E2E Distances

In [ ]:
disc_center = traj.xyz[0, a_sele, :].mean(axis=0)

e2e_dists = []
for i in range(traj.n_chains):
    ch_a = traj.top.chain(i).n_atoms
    arm_end = traj.xyz[:, i*ch_a+ch_a-1, :]
    e2e_dists.append(10.*np.linalg.norm(arm_end-disc_center, axis=1))

In [ ]:
plt.figure(facecolor='w')
for i in range(traj.n_chains):    
    plt.plot(range(traj.n_frames), e2e_dists[i])
    
plt.xlabel('Frame')
plt.ylabel('Arm End to Disc Center [$\mathrm{\AA}$]')

plt.axhline(y=25, color='k', ls='--')
plt.axhline(y=15, color='k', ls='--')

plt.annotate("Disc Radius", xy=(250, 25), xytext=(0,-15),
             textcoords='offset pixels', fontsize=14)
plt.annotate("Disc Half Thickness", xy=(250, 15), xytext=(0,-15),
             textcoords='offset pixels', fontsize=14)

plt.show()

# ARM-Disc Charge Contacts

In [ ]:
cutoff = 10.

arms_in_contact = []

for i in range(traj.n_chains):
    arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
    pair_list = np.array([(i, j) for i in arm_charges for j in base_sele])
    
    dists = 10.*md.compute_distances(traj, pair_list, periodic=False, opt=True)
    is_contact = dists < cutoff
    arms_in_contact.append(is_contact.any(axis=1))
arms_in_contact = np.stack(arms_in_contact, axis=1).sum(axis=1)

In [ ]:
dists.shape

In [ ]:
plt.figure(facecolor='w')    
plt.plot(range(traj.n_frames), arms_in_contact)
    
plt.xlabel('Frame')
plt.ylabel('# of Arm-Disc Opposite Charge Contacts')
plt.show()

In [ ]:
cutoff = 10.

n = 3
m = -(-n_rep//n)
nrows = int(ceil(n_rep/n))
fig0 = plt.figure(figsize=(5*n,3.5*m))

for rep in range(n_rep):
    plt.subplot(nrows, n, rep+1)
    
    traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
    traj = mu.load_upside_traj(traj_fn, stride=1)
    
    arms_in_contact = []
    for i in range(traj.n_chains):
        arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
        pair_list = np.array([(i, j) for i in arm_charges for j in base_sele])

        dists = 10.*md.compute_distances(traj, pair_list, periodic=False, opt=True)
        is_contact = dists < cutoff
        arms_in_contact.append(is_contact.any(axis=1))
    arms_in_contact = np.stack(arms_in_contact, axis=1).sum(axis=1)
    
    plt.plot(range(traj.n_frames), arms_in_contact)
    plt.title("Replica {}".format(rep))

fig0.text(0., 0.5, '# of Arm-Disc Opposite Charge Contacts', fontsize=28, ha='center', va='center', rotation='vertical')
fig0.text(0.5, 0., 'Frame', fontsize=28, ha='center', va='center')
plt.tight_layout()

# ARM-Disc Side Preferance

In [ ]:
top_contacts = []
bottom_contacts = []
top_charges=[]
bottom_charges=[]

for i in range(traj.n_chains):
    top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
    bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())
    # print(top_charges)
    # print(bottom_charges)

for i in range(traj.n_chains):
    arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
    top_pair_list = np.array([(i, j) for i in arm_charges for j in top_charges])
    bottom_pair_list = np.array([(i, j) for i in arm_charges for j in bottom_charges])
    
    top_dists = 10.*md.compute_distances(traj, top_pair_list, periodic=False, opt=True)
    bottom_dists = 10.*md.compute_distances(traj, bottom_pair_list, periodic=False, opt=True)
    is_contact = top_dists < cutoff
    top_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_dists < cutoff
    bottom_contacts.append(is_contact.any(axis=1))

# print(len(top_contacts))
# print(top_contacts)
top_contacts = np.stack(top_contacts, axis=1).sum(axis=1)
bottom_contacts = np.stack(bottom_contacts, axis=1).sum(axis=1)
    
plt.figure(facecolor='w')    
plt.plot(range(traj.n_frames), top_contacts,color="b")
plt.plot(range(traj.n_frames), bottom_contacts*-1,color="r")

    
plt.xlabel('Frame')
plt.ylabel('# of Arm-Disc Opposite Charge Contacts')
plt.show()

print("Total number of top contacts: {}".format(np.sum(top_contacts)))
print("Total number of bottom contacts: {}".format(np.sum(bottom_contacts)))

In [ ]:
asdas=[3,2,1]
print(asdas.append(['x','y','z']))
print(str(asdas))

In [ ]:
top_contacts = []
bottom_contacts = []

for i in range(traj.n_chains):
    top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
    bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())
    
for i in range(traj.n_chains):
    bottom_charges = traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65))
    top_pair_list = np.array([(i, j) for i in arm_charges for j in top_charges])
    bottom_pair_list = np.array([(i, j) for i in arm_charges for j in bottom_charges])
    
    top_dists = 10.*md.compute_distances(traj, top_pair_list, periodic=False, opt=True)
    bottom_dists = 10.*md.compute_distances(traj, bottom_pair_list, periodic=False, opt=True)
    is_contact = top_dists < cutoff
    top_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_dists < cutoff
    bottom_contacts.append(is_contact.any(axis=1))

top_contacts = np.stack(top_contacts, axis=1).sum(axis=1)
bottom_contacts = np.stack(bottom_contacts, axis=1).sum(axis=1)

In [ ]:
top_contacts = []
bottom_contacts = []

top_charges=[]
bottom_charges=[]

for i in range(traj.n_chains):
    top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
    bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())
    
for i in range(traj.n_chains):
    arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
    top_pair_list = np.array([(i, j) for i in arm_charges for j in top_charges])
    bottom_pair_list = np.array([(i, j) for i in arm_charges for j in bottom_charges])
    
    top_dists = 10.*md.compute_distances(traj, top_pair_list, periodic=False, opt=True)
    bottom_dists = 10.*md.compute_distances(traj, bottom_pair_list, periodic=False, opt=True)
    is_contact = top_dists < cutoff
    top_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_dists < cutoff
    bottom_contacts.append(is_contact.any(axis=1))

top_contacts = np.stack(top_contacts, axis=1).sum(axis=1)
bottom_contacts = np.stack(bottom_contacts, axis=1).sum(axis=1)

top_contacts_kde = []
for i in range(0,len(top_contacts)):
    if top_contacts[i] == 1:
        top_contacts_kde.append(i)
    if top_contacts[i] == 2:
        top_contacts_kde.append(i)
        top_contacts_kde.append(i)
    if top_contacts[i] == 3:
        top_contacts_kde.append(i)
        top_contacts_kde.append(i)
        top_contacts_kde.append(i)
    if bottom_contacts[i] == 4:
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)      
        bottom_contacts_kde.append(i)    
        
bottom_contacts_kde = []
for i in range(0,len(bottom_contacts)):
    if bottom_contacts[i] == 1:
        bottom_contacts_kde.append(i)
    if bottom_contacts[i] == 2:
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)
    if bottom_contacts[i] == 3:
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)
    if bottom_contacts[i] == 4:
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)
        bottom_contacts_kde.append(i)      
        bottom_contacts_kde.append(i)

plt.figure(facecolor='w')    
sb.kdeplot(top_contacts_kde,gridsize=1000,shade=True,color="b")
sb.kdeplot(bottom_contacts_kde,gridsize=1000,shade=True,color="r")

    
plt.xlabel('Frame')
plt.ylabel('Density')
plt.legend()
plt.show()

print("Total number of top contacts: {}".format(np.sum(top_contacts)))
print("Total number of bottom contacts: {}".format(np.sum(bottom_contacts)))

In [ ]:
cutoff = 10.

n = 3
m = -(-n_rep//n)
nrows = int(ceil(n_rep/n))
fig0 = plt.figure(figsize=(5*n,3.5*m))

contact_count=[]
top_contact_count=0
bottom_contact_count=0
top_contact_table=np.zeros(5)
bottom_contact_table=np.zeros(5)



for rep in range(n_rep):
    plt.subplot(nrows, n, rep+1)
    
    traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
    traj = mu.load_upside_traj(traj_fn, stride=1)
    
    top_contacts = []
    bottom_contacts = []
    
    
    top_charges =[]
    bottom_charges=[]
    
    for i in range(traj.n_chains):
        top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
        bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())
    
    for i in range(traj.n_chains):
        arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
        top_pair_list = np.array([(i, j) for i in arm_charges for j in top_charges])
        bottom_pair_list = np.array([(i, j) for i in arm_charges for j in bottom_charges])
        
        top_dists = 10.*md.compute_distances(traj, top_pair_list, periodic=False, opt=True)
        bottom_dists = 10.*md.compute_distances(traj, bottom_pair_list, periodic=False, opt=True)
        is_contact = top_dists < cutoff
        top_contacts.append(is_contact.any(axis=1))
        is_contact = bottom_dists < cutoff
        bottom_contacts.append(is_contact.any(axis=1))

    top_contacts = np.stack(top_contacts, axis=1).sum(axis=1)
    bottom_contacts = np.stack(bottom_contacts, axis=1).sum(axis=1)
    
    rep_top_contact_table=np.bincount(top_contacts)
    rep_bottom_contact_table=np.bincount(bottom_contacts)
    
    top_contact_table += np.pad(rep_top_contact_table,(0,5-rep_top_contact_table.size),'constant')
    bottom_contact_table += np.pad(rep_bottom_contact_table,(0,5-rep_bottom_contact_table.size),'constant')
  
    plt.plot(range(traj.n_frames), top_contacts,color="b")
    plt.plot(range(traj.n_frames), bottom_contacts*-1,color="r")
    plt.title("Replica {}".format(rep+1))
    
    print("{} top contacts in replica {}".format(np.sum(top_contacts),rep))
    print("{} bottom contacts in replica {}".format(np.sum(bottom_contacts),rep))
    
    contact_count.append(np.sum(top_contacts))
    contact_count.append(np.sum(-1*bottom_contacts))
    
    top_contact_count += np.sum(top_contacts)
    bottom_contact_count += np.sum(bottom_contacts)

# expected_val = top_contact_count + bottom_contact_count * 0.5
# chi_sqr = ((top_contact_count - expected_val)**2 + ((bottom_contact_count - expected_val)**2))/expected_val
    
# contact_count=np.array(contact_count)
# print("The mean number of contacts across each replica is {} and the standard deviation is {}.".format(np.mean(contact_count),np.std(contact_count)))
# print("The total number of top contacts is {} and the total number of botom contacts is {}. The chi-squared is {}.".format(top_contact_count,bottom_contact_count,chi_sqr))
# print("The p-value is {}".format(1-stats.chi2.cdf(chi_sqr,1)))

fig0.text(0., 0.5, '# of Arm-Disc Opposite Charge Contacts', fontsize=28, ha='center', va='center', rotation='vertical')
fig0.text(0.5, 0., 'Frame', fontsize=28, ha='center', va='center')
plt.tight_layout()

In [ ]:
col_names=["No Contact","Single Contact","Double Contact","Triple Contact","Quadruple Contact"]
obs_data = {
    '': col_names,
    'Top Contact Count': top_contact_table,
    'Bottom Contact Count': bottom_contact_table,
}
pd.DataFrame(obs_data)

In [ ]:
cutoff = 10.

n = 3
m = -(-n_rep//n)
nrows = int(ceil(n_rep/n))
fig0 = plt.figure(figsize=(5*n,3.5*m))

contact_count=[]
top_contact_count=0
bottom_contact_count=0
top_contact_table=np.zeros(5)
bottom_contact_table=np.zeros(5)



for rep in range(n_rep):
    plt.subplot(nrows, n, rep+1)
    
    traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
    traj = mu.load_upside_traj(traj_fn, stride=1)
    
    top_contacts = []
    bottom_contacts = []
    
    arm_charges =[]
    bottom_charges=[]
    
    for i in range(traj.n_chains):
        top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
        bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())
    
    for i in range(traj.n_chains):
        arm_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(i, i*102+66))
        top_pair_list = np.array([(i, j) for i in arm_charges for j in top_charges])
        bottom_pair_list = np.array([(i, j) for i in arm_charges for j in bottom_charges])
        
        top_dists = 10.*md.compute_distances(traj, top_pair_list, periodic=False, opt=True)
        bottom_dists = 10.*md.compute_distances(traj, bottom_pair_list, periodic=False, opt=True)
        is_contact = top_dists < cutoff
        top_contacts.append(is_contact.any(axis=1))
        is_contact = bottom_dists < cutoff
        bottom_contacts.append(is_contact.any(axis=1))

    top_contacts = np.stack(top_contacts, axis=1).sum(axis=1)
    bottom_contacts = np.stack(bottom_contacts, axis=1).sum(axis=1)
    
    top_contacts_kde = []
    for i in range(0,len(top_contacts)):
        if top_contacts[i] == 1:
            top_contacts_kde.append(i)
        if top_contacts[i] == 2:
            top_contacts_kde.append(i)
            top_contacts_kde.append(i)
        if top_contacts[i] == 3:
            top_contacts_kde.append(i)
            top_contacts_kde.append(i)
            top_contacts_kde.append(i)
        if bottom_contacts[i] == 4:
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)      
            bottom_contacts_kde.append(i)    

    bottom_contacts_kde = []
    for i in range(0,len(bottom_contacts)):
        if bottom_contacts[i] == 1:
            bottom_contacts_kde.append(i)
        if bottom_contacts[i] == 2:
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)
        if bottom_contacts[i] == 3:
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)
        if bottom_contacts[i] == 4:
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)
            bottom_contacts_kde.append(i)      
            bottom_contacts_kde.append(i)
   
    sb.kdeplot(top_contacts_kde,gridsize=1000,shade=True,color="b")
    sb.kdeplot(bottom_contacts_kde,gridsize=1000,shade=True,color="r")
    plt.title("Replica {}".format(rep+1))
    
plt.xlabel('Frame')
plt.ylabel('Density')
plt.show()

In [ ]:
col_names=["No Contact","Single Contact","Double Contact","Triple Contact","Quadruple Contact"]
thr_data = {
    '': col_names,
    'Predicted Top Contact': [18750],
    'Predicted Bottom Contact': [],
}
# pd.DataFrame(thr_data)

In [ ]:
str([1,5,col_names])

# Arm Sticking

In [ ]:
cutoff = 10

top_charges=[]
bottom_charges=[]

for i in range(traj.n_chains):
    top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
    bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())

arm_1_top_contacts = []
arm_2_top_contacts = []
arm_3_top_contacts = []
arm_4_top_contacts = []
arm_5_top_contacts = []
arm_6_top_contacts = []

arm_1_bottom_contacts = []
arm_2_bottom_contacts = []
arm_3_bottom_contacts = []
arm_4_bottom_contacts = []
arm_5_bottom_contacts = []
arm_6_bottom_contacts = []

arm_1_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(0, 0*102+66))
arm_2_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(1, 1*102+66))
arm_3_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(2, 2*102+66))
arm_4_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(3, 3*102+66))
arm_5_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(4, 4*102+66))
arm_6_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(5, 5*102+66))
    
top_pair_1_list = np.array([(i, j) for i in arm_1_charges for j in top_charges])
top_pair_2_list = np.array([(i, j) for i in arm_2_charges for j in top_charges])
top_pair_3_list = np.array([(i, j) for i in arm_3_charges for j in top_charges])
top_pair_4_list = np.array([(i, j) for i in arm_4_charges for j in top_charges])
top_pair_5_list = np.array([(i, j) for i in arm_5_charges for j in top_charges])
top_pair_6_list = np.array([(i, j) for i in arm_6_charges for j in top_charges])

bottom_pair_1_list = np.array([(i, j) for i in arm_1_charges for j in bottom_charges])
bottom_pair_2_list = np.array([(i, j) for i in arm_2_charges for j in bottom_charges])
bottom_pair_3_list = np.array([(i, j) for i in arm_3_charges for j in bottom_charges])
bottom_pair_4_list = np.array([(i, j) for i in arm_4_charges for j in bottom_charges])
bottom_pair_5_list = np.array([(i, j) for i in arm_5_charges for j in bottom_charges])
bottom_pair_6_list = np.array([(i, j) for i in arm_6_charges for j in bottom_charges])
    
top_1_dists = 10.*md.compute_distances(traj, top_pair_1_list, periodic=False, opt=True)
top_2_dists = 10.*md.compute_distances(traj, top_pair_2_list, periodic=False, opt=True)
top_3_dists = 10.*md.compute_distances(traj, top_pair_3_list, periodic=False, opt=True)
top_4_dists = 10.*md.compute_distances(traj, top_pair_4_list, periodic=False, opt=True)
top_5_dists = 10.*md.compute_distances(traj, top_pair_5_list, periodic=False, opt=True)
top_6_dists = 10.*md.compute_distances(traj, top_pair_6_list, periodic=False, opt=True)
    
bottom_1_dists = 10.*md.compute_distances(traj, bottom_pair_1_list, periodic=False, opt=True)
bottom_2_dists = 10.*md.compute_distances(traj, bottom_pair_2_list, periodic=False, opt=True)
bottom_3_dists = 10.*md.compute_distances(traj, bottom_pair_3_list, periodic=False, opt=True)
bottom_4_dists = 10.*md.compute_distances(traj, bottom_pair_4_list, periodic=False, opt=True)
bottom_5_dists = 10.*md.compute_distances(traj, bottom_pair_5_list, periodic=False, opt=True)
bottom_6_dists = 10.*md.compute_distances(traj, bottom_pair_6_list, periodic=False, opt=True)
    
is_contact = top_1_dists < cutoff
arm_1_top_contacts.append(is_contact.any(axis=1))
is_contact = top_2_dists < cutoff
arm_2_top_contacts.append(is_contact.any(axis=1))
is_contact = top_3_dists < cutoff
arm_3_top_contacts.append(is_contact.any(axis=1))
is_contact = top_4_dists < cutoff
arm_4_top_contacts.append(is_contact.any(axis=1))
is_contact = top_5_dists < cutoff
arm_5_top_contacts.append(is_contact.any(axis=1))
is_contact = top_6_dists < cutoff
arm_6_top_contacts.append(is_contact.any(axis=1))

is_contact = bottom_1_dists < cutoff
arm_1_bottom_contacts.append(is_contact.any(axis=1))
is_contact = bottom_2_dists < cutoff
arm_2_bottom_contacts.append(is_contact.any(axis=1))
is_contact = bottom_3_dists < cutoff
arm_3_bottom_contacts.append(is_contact.any(axis=1))
is_contact = bottom_4_dists < cutoff
arm_4_bottom_contacts.append(is_contact.any(axis=1))
is_contact = bottom_5_dists < cutoff
arm_5_bottom_contacts.append(is_contact.any(axis=1))
is_contact = bottom_6_dists < cutoff
arm_6_bottom_contacts.append(is_contact.any(axis=1))

plt.figure(facecolor='w')
stkplt_labels=[]
for k in ['top','bottom']:
    for m in range(6):
        stkplt_labels.append("{}. {} arm".format(m+1,k))
plt.stackplot(range(traj.n_frames), arm_1_top_contacts[0],arm_2_top_contacts[0],arm_3_top_contacts[0],arm_4_top_contacts[0],arm_5_top_contacts,arm_6_top_contacts[0],arm_1_bottom_contacts[0]*-1,arm_2_bottom_contacts[0]*-1,arm_3_bottom_contacts[0]*-1,arm_4_bottom_contacts[0]*-1,arm_5_bottom_contacts[0]*-1,arm_6_bottom_contacts[0]*-1,labels=stkplt_labels,baseline="zero")
plt.legend(title="legend",prop={'size':10},loc="center left")
    
plt.xlabel('Frame',loc="center")
plt.ylabel('# of Arm-Disc Opposite Charge Contacts')
plt.show()
    


In [ ]:
cutoff = 1

n = 3
m = -(-n_rep//n)
nrows = int(ceil(n_rep/n))
fig0 = plt.figure(figsize=(5*n,3.5*m))


stkplt_labels=[]
for k in ['top','bottom']:
    for m in range(5):
        stkplt_labels.append("{}. {} arm".format(m+1,k))

for rep in range(n_rep):
    plt.subplot(nrows, n, rep+1)
    
    traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
    traj = mu.load_upside_traj(traj_fn, stride=1)
    
    top_charges=[]
    bottom_charges=[]
    
    for i in range(traj.n_chains):
        top_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i, i*102+2,i*102+15,i*102+16,i*102+55)).tolist())
        bottom_charges.extend(traj.top.select("chainid {} and ((resid {} or resid {}) or (resid {} or resid {})) and name CA".format(i,i*102+18,i*102+30,i*102+46,i*102+65)).tolist())

    arm_1_top_contacts = []
    arm_2_top_contacts = []
    arm_3_top_contacts = []
    arm_4_top_contacts = []
    arm_5_top_contacts = []
    arm_6_top_contacts = []

    arm_1_bottom_contacts = []
    arm_2_bottom_contacts = []
    arm_3_bottom_contacts = []
    arm_4_bottom_contacts = []
    arm_5_bottom_contacts = []
    arm_6_bottom_contacts = []

    arm_1_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(0, 0*102+66))
    arm_2_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(1, 1*102+66))
    arm_3_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(2, 2*102+66))
    arm_4_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(3, 3*102+66))
    arm_5_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(4, 4*102+66))
    arm_6_charges = traj.top.select("chainid {} and (resid > {}) and (resname ASP or resname GLU) and name CA".format(5, 5*102+66))

    top_pair_1_list = np.array([(i, j) for i in arm_1_charges for j in top_charges])
    top_pair_2_list = np.array([(i, j) for i in arm_2_charges for j in top_charges])
    top_pair_3_list = np.array([(i, j) for i in arm_3_charges for j in top_charges])
    top_pair_4_list = np.array([(i, j) for i in arm_4_charges for j in top_charges])
    top_pair_5_list = np.array([(i, j) for i in arm_5_charges for j in top_charges])
    top_pair_6_list = np.array([(i, j) for i in arm_6_charges for j in top_charges])

    bottom_pair_1_list = np.array([(i, j) for i in arm_1_charges for j in bottom_charges])
    bottom_pair_2_list = np.array([(i, j) for i in arm_2_charges for j in bottom_charges])
    bottom_pair_3_list = np.array([(i, j) for i in arm_3_charges for j in bottom_charges])
    bottom_pair_4_list = np.array([(i, j) for i in arm_4_charges for j in bottom_charges])
    bottom_pair_5_list = np.array([(i, j) for i in arm_5_charges for j in bottom_charges])
    bottom_pair_6_list = np.array([(i, j) for i in arm_6_charges for j in bottom_charges])

    top_1_dists = 10.*md.compute_distances(traj, top_pair_1_list, periodic=False, opt=True)
    top_2_dists = 10.*md.compute_distances(traj, top_pair_2_list, periodic=False, opt=True)
    top_3_dists = 10.*md.compute_distances(traj, top_pair_3_list, periodic=False, opt=True)
    top_4_dists = 10.*md.compute_distances(traj, top_pair_4_list, periodic=False, opt=True)
    top_5_dists = 10.*md.compute_distances(traj, top_pair_5_list, periodic=False, opt=True)
    top_6_dists = 10.*md.compute_distances(traj, top_pair_6_list, periodic=False, opt=True)

    bottom_1_dists = 10.*md.compute_distances(traj, bottom_pair_1_list, periodic=False, opt=True)
    bottom_2_dists = 10.*md.compute_distances(traj, bottom_pair_2_list, periodic=False, opt=True)
    bottom_3_dists = 10.*md.compute_distances(traj, bottom_pair_3_list, periodic=False, opt=True)
    bottom_4_dists = 10.*md.compute_distances(traj, bottom_pair_4_list, periodic=False, opt=True)
    bottom_5_dists = 10.*md.compute_distances(traj, bottom_pair_5_list, periodic=False, opt=True)
    bottom_6_dists = 10.*md.compute_distances(traj, bottom_pair_6_list, periodic=False, opt=True)

    is_contact = top_1_dists < cutoff
    arm_1_top_contacts.append(is_contact.any(axis=1))
    is_contact = top_2_dists < cutoff
    arm_2_top_contacts.append(is_contact.any(axis=1))
    is_contact = top_3_dists < cutoff
    arm_3_top_contacts.append(is_contact.any(axis=1))
    is_contact = top_4_dists < cutoff
    arm_4_top_contacts.append(is_contact.any(axis=1))
    is_contact = top_5_dists < cutoff
    arm_5_top_contacts.append(is_contact.any(axis=1))
    is_contact = top_6_dists < cutoff
    arm_6_top_contacts.append(is_contact.any(axis=1))

    is_contact = bottom_1_dists < cutoff
    arm_1_bottom_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_2_dists < cutoff
    arm_2_bottom_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_3_dists < cutoff
    arm_3_bottom_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_4_dists < cutoff
    arm_4_bottom_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_5_dists < cutoff
    arm_5_bottom_contacts.append(is_contact.any(axis=1))
    is_contact = bottom_6_dists < cutoff
    arm_6_bottom_contacts.append(is_contact.any(axis=1))
    
    plt.stackplot(range(traj.n_frames), arm_1_top_contacts[0],arm_2_top_contacts[0],arm_3_top_contacts[0],arm_4_top_contacts[0],arm_5_top_contacts,arm_6_top_contacts[0],arm_1_bottom_contacts[0]*-1,arm_2_bottom_contacts[0]*-1,arm_3_bottom_contacts[0]*-1,arm_4_bottom_contacts[0]*-1,arm_5_bottom_contacts[0]*-1,arm_6_bottom_contacts[0]*-1,labels=stkplt_labels,baseline="zero")
    plt.title("Replica {}".format(rep+1))

#plt.legend(title="legend",prop={'size':8},loc="center left")
#fig0.text(0., 0.5, '# of Arm-Disc Opposite Charge Contacts', fontsize=28, ha='center', va='center', rotation='vertical')
#fig0.text(0.5, 0., 'Frame', fontsize=28, ha='center', va='center')
plt.tight_layout()

In [ ]:
bob=['x','y','z']
bob=('x','y','z')

[[bob[i],i,'temp'] for i in range(3)]

# Dipole

In [ ]:
#use mdtraj package to find dipole at each frame then find average dipole across all frames

#creating array of charges
trajCharges=np.zeros(traj.n_atoms)
for i in acid_sele:
    trajCharges[i]=-1
for i in base_sele:
    trajCharges[i]=1
#(1000,3) array with dipole moment at each frame
trajDipoleMoments=md.dipole_moments(traj,trajCharges)
#calculating stats
print("mean dipole: {}".format(np.mean(trajDipoleMoments,axis=0)))
print("std dipole: {}".format(np.std(trajDipoleMoments,axis=0)))
trajDipoleMomentMagnitude = []
for i in range (0,999):
    trajDipoleMomentMagnitude.append(np.linalg.norm(trajDipoleMoments[i],ord=2))
print("mean magnitude of dipole: {}".format(np.mean(trajDipoleMomentMagnitude)))
print("std of magnitude of dipole: {}".format(np.std(trajDipoleMomentMagnitude)))

# Rg

## Accross single replica trajectory

In [ ]:
rep = 0

traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
traj = mu.load_upside_traj(traj_fn, stride=1)

rg = 10.*md.compute_rg(traj)
x = np.arange(traj.n_frames)

plt.scatter(x, rg)
plt.xlabel("Rg [$\AA$]")
plt.ylabel("Frame") #change````````````````````````````````````````````````````````````````````````````````````````````
plt.show()

## Melting curves

In [ ]:
equil_frac = 1/4 # fraction of frames to discard for equilibration

temps = np.zeros(n_rep)
rg_mu = np.zeros(n_rep)
rg_std = np.zeros(n_rep)

for rep in range(n_rep):
    traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
    
    with tb.open_file(traj_fn) as t:
        temps[rep] = t.root.output.temperature[0]
        
    traj = mu.load_upside_traj(traj_fn, stride=1)
    rg = 10.*md.compute_rg(traj)[int(traj.n_frames*equil_frac):]
    rg_mu[rep] = rg.mean()
    rg_std[rep] = rg.std()
    
plt.errorbar(temps, rg_mu, yerr=rg_std, fmt='o--')
plt.xlabel("Temperature [kT]")
plt.ylabel("<Rg> [$\AA$]")
plt.show()

# RMSD

## Accross single replica trajectory

In [ ]:
rep = 0

traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
traj = mu.load_upside_traj(traj_fn, stride=1)

# Choose a reference structure. Either load from a file...
ref_fn = base_dir + "{}.pdb".format(pdb_id)
ref = md.load(ref_fn)
# Or pick a frame in the trajectory of interest. Uncomment the following line for that
# native = traj[0]

sele_CA_ref = ref.top.select("name CA") # Select CA for reference
sele_CA_traj = traj.top.select("name CA") # Select CA for traj

rmsd = 10.*md.rmsd(traj.atom_slice(sele_CA_traj), ref.atom_slice(sele_CA_ref))
x = np.arange(traj.n_frames)

plt.scatter(x, rmsd)
plt.xlabel("RMSD [$\AA$]")
plt.ylabel("Frame")
plt.show()

## Distribution

In [ ]:
from scipy.stats import gaussian_kde

rep = 0
equil_frac = 1/4 # fraction of frames to discard for equilibration

traj_fn = output_dir + "{}.{}.up".format(pdb_id, rep)
traj = mu.load_upside_traj(traj_fn, stride=1)

# Choose a reference structure. Either load from a file...
ref_fn = base_dir + "{}.pdb".format(pdb_id)
ref = md.load(ref_fn)
# Or pick a frame in the trajectory of interest. Uncomment the following line for that
# native = traj[0]

sele_CA_ref = ref.top.select("name CA") # Select CA for reference
sele_CA_traj = traj.top.select("name CA") # Select CA for traj

rmsd = 10.*md.rmsd(traj.atom_slice(sele_CA_traj), ref.atom_slice(sele_CA_ref))[int(traj.n_frames*equil_frac):]
dat_min = rmsd.min()
dat_max = rmsd.max()
x = np.linspace(dat_min, dat_max, num=int((dat_max-dat_min)/8.*(128+1)))
kernel = gaussian_kde(rmsd)
y = kernel(x)

plt.plot(x, y)
plt.xlim(left=0)
plt.xlabel("RMSD [$\AA$]")
plt.ylabel("Frequency")
plt.show()